In [1]:
import pandas as pd
import numpy as np
import requests
from io import StringIO

# Data Collection

In [2]:
# API endpoint and headers
url = 'https://my.api.mockaroo.com/phone_inventory.json'
headers = {'X-API-Key': '1cf59630'}

# Perform GET request
response = requests.get(url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    # The response is in text format
    data_text = response.text

    # Convert text into a pandas DataFrame
    # Since the data is in CSV format, we use StringIO to convert it into a file-like object
    data = StringIO(data_text)
    df = pd.read_csv(data)

    # Display the DataFrame
    print(df)
else:
    print(f"Failed to retrieve data. Status code: {response.status_code}")


     product_name        date  stock_level  sales
0             HTC   7/10/2024           88    7.0
1         OnePlus    1/9/2024          187  102.0
2          Huawei  10/16/2024          190   16.0
3          Lenovo   3/22/2024           28   28.0
4     Sony Xperia    9/5/2024           13    9.0
..            ...         ...          ...    ...
995  Google Pixel         NaN           89   45.0
996  Google Pixel  12/25/2024          154   32.0
997        Xiaomi    9/2/2024          121  121.0
998            LG  11/24/2024          174    2.0
999            LG   8/20/2024           75    NaN

[1000 rows x 4 columns]


# Data Exploration

In [3]:
df.head()

,product_name,date,stock_level,sales
0,HTC,7/10/2024,88,7.0
1,OnePlus,1/9/2024,187,102.0
2,Huawei,10/16/2024,190,16.0
3,Lenovo,3/22/2024,28,28.0
4,Sony Xperia,9/5/2024,13,9.0


In [4]:
df.columns

Index(['product_name', 'date', 'stock_level', 'sales'], dtype='object')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   product_name  1000 non-null   object 
 1   date          940 non-null    object 
 2   stock_level   1000 non-null   int64  
 3   sales         940 non-null    float64
dtypes: float64(1), int64(1), object(2)
memory usage: 31.4+ KB


In [6]:
print(df.dtypes)

product_name     object
date             object
stock_level       int64
sales           float64
dtype: object


In [7]:
df['product_name'].unique()

array(['HTC', 'OnePlus', 'Huawei', 'Lenovo', 'Sony Xperia', 'ZTE', 'LG',
       'Alcatel', 'Google Pixel', 'Xiaomi', 'BlackBerry', 'Nokia',
       'Motorola', 'Samsung Galaxy', 'iPhone'], dtype=object)

In [8]:
df['product_name'].nunique()

15

# Data Cleaning

### 1. Remove Missing or Invalid Data

The method of handling missing data in each column can be determined by the percentage of missing values.
* For columns with a high proportion of missing values, consider imputing the data using the mean or mode.
* If the percentage of missing values is low, it may be more effective to simply delete the incomplete entries.

In [9]:
#number and percentage of missing elements of each columns
total= df.isnull().sum()
percent = (df.isnull().sum()/df.isnull().count())
missing = pd.concat([total, percent*100], axis=1, keys=['Total', 'Percent'])
missing

,Total,Percent
product_name,0,0.0
date,60,6.0
stock_level,0,0.0
sales,60,6.0


In [10]:
for column in df.columns:
    if missing['Percent'][column] > 30:  # If more than 30% of the column is missing
        # Impute with the mode for categorical columns or mean for numerical columns
        if df[column].dtype == 'object':  # For categorical columns
            df[column].fillna(df[column].mode()[0], inplace=True)
        else:  # For numerical columns
            df[column].fillna(df[column].mean(), inplace=True)
    else:
        # Drop rows with missing values if less than 30% are missing
        df.dropna(subset=[column], inplace=True)

df.head()

,product_name,date,stock_level,sales
0,HTC,7/10/2024,88,7.0
1,OnePlus,1/9/2024,187,102.0
2,Huawei,10/16/2024,190,16.0
3,Lenovo,3/22/2024,28,28.0
4,Sony Xperia,9/5/2024,13,9.0


In [11]:
df.isnull().sum()

,0
product_name,0
date,0
stock_level,0
sales,0


### 2. Checking for Duplicate Rows


In [12]:
# Check for duplicate rows
df = df.drop_duplicates()
duplicates = df.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")

Number of duplicate rows: 0


### 3.Fix Columns Types

In [13]:
# Convert 'date' column to datetime format
df['date'] = pd.to_datetime(df['date'], format='%m/%d/%Y')

# Convert 'sales' column to integer type
df['sales'] = df['sales'].astype(int)

# Print the data types of each column
print(df.dtypes)

product_name            object
date            datetime64[ns]
stock_level              int64
sales                    int64
dtype: object


### 4. Outlier Detection

In [14]:
from scipy import stats
# Select numeric columns
numeric_columns = df[['stock_level', 'sales']]

# Calculate Z-scores
z_scores = stats.zscore(numeric_columns)
abs_z_scores = np.abs(z_scores)
outliers = df[(abs_z_scores >= 5).any(axis=1)]

# Show the outliers
print(outliers)

Empty DataFrame
Columns: [product_name, date, stock_level, sales]
Index: []


In [15]:
# Check if sales are less than stock_level
sales_greater_or_equal_stock = df[df['sales'] > df['stock_level']]
print(sales_greater_or_equal_stock)

Empty DataFrame
Columns: [product_name, date, stock_level, sales]
Index: []


### 5. Feature Engineering

In [16]:
# Feature engineering (e.g., extracting date-related features)
df['Year'] = df['date'].dt.year
df['Month'] = df['date'].dt.month
df['Day'] = df['date'].dt.day

In [17]:
df.head()

,product_name,date,stock_level,sales,Year,Month,Day
0,HTC,2024-07-10,88,7,2024,7,10
1,OnePlus,2024-01-09,187,102,2024,1,9
2,Huawei,2024-10-16,190,16,2024,10,16
3,Lenovo,2024-03-22,28,28,2024,3,22
4,Sony Xperia,2024-09-05,13,9,2024,9,5


In [18]:
# Convert the cleaned DataFrame to CSV format
csv_file_path = '/content/cleaned_data.csv'
df.to_csv(csv_file_path, index=False)